<a href="https://colab.research.google.com/github/sooonsyk/ESAA/blob/main/cluster_mvm_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
DF_2022=pd.read_csv('/content/drive/MyDrive/DF_2022_KNN.csv')
col = ['MESURE_AGE_CO','SEXDSTN_FLAG_CD','MESURE_IEM_001_VALUE','MESURE_IEM_002_VALUE','MESURE_IEM_003_VALUE','MESURE_IEM_005_VALUE','MESURE_IEM_006_VALUE','MESURE_IEM_012_VALUE','MESURE_IEM_018_VALUE']
df_2022 = DF_2022[col]
df_2022 = pd.get_dummies(data=df_2022)

In [4]:
from sklearn.cluster import KMeans

In [5]:
kmeans = KMeans(n_clusters=7, init='k-means++', max_iter=300, random_state=42)
kmeans.fit(df_2022)
df_2022['cluster'] = kmeans.labels_
df_2022['MVM_PRSCRPTN_CN'] = DF_2022['df_2022[, 16]']
df_2022['cluster'].value_counts()

0    31683
2    26071
3    19379
1    19204
4    17492
6        3
5        2
Name: cluster, dtype: int64

In [18]:
df_2022_c0 = df_2022[df_2022['cluster']==0]
df_2022_c1 = df_2022[df_2022['cluster']==1]
df_2022_c2 = df_2022[df_2022['cluster']==2]
df_2022_c3 = df_2022[df_2022['cluster']==3]
df_2022_c4 = df_2022[df_2022['cluster']==4]
df_2022_c5 = df_2022[df_2022['cluster']==5]
df_2022_c6 = df_2022[df_2022['cluster']==6]

In [7]:
def mvm_list(data):
  data_mvm = data['MVM_PRSCRPTN_CN'].dropna(axis=0)

  data_ispre = data_mvm[data_mvm.str.startswith('준')]  #준비운동 포함
  data_nopre = data_mvm[data_mvm.str.startswith('준')==False] # 준비운동 포함 안 함

  ispre_nomain = data_ispre.str.split('본운동:',expand=True).reset_index(drop=True)[:][0]
  ispre_ismain = data_ispre.str.split('본운동:',expand=True).reset_index(drop=True)[:][1]
  ispre_ismain.dropna(axis=0, inplace=True)

  ispre_nomain_noend = ispre_nomain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][0] 
  ispre_nomain_isend = ispre_nomain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][1]
  ispre_nomain_isend.dropna(axis=0, inplace=True)

  ispre_ismain_noend = ispre_ismain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][0] 
  ispre_ismain_isend = ispre_ismain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][1] 
  ispre_ismain_isend.dropna(axis=0, inplace=True)

  nopre_noend = data_nopre.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][0]
  nopre_isend = data_nopre.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][1]
  nopre_isend.dropna(axis=0,inplace=True)

  #준비운동 리스트 만들기
  ispre_nomain_noend = ispre_nomain_noend.str.replace('준비운동:','')
  ispre_nomain_noend = ispre_nomain_noend.str.replace(' / ','')
  pre_data = ispre_nomain_noend.str.split(',')
  pre_list = []
  for i in range(pre_data.shape[0]):
    for j in range(len(pre_data[i])):
      pre_list.append(pre_data[i][j])
  pre_list = pd.DataFrame(pre_list)

  #본운동 리스트 만들기
  nopre_noend = nopre_noend.str.replace('본운동:','')
  nopre_noend = nopre_noend.str.replace(' / ','')
  nopre_noend = nopre_noend.str.split(',')
  ispre_ismain_noend = ispre_ismain_noend.str.replace(' / ','')
  ispre_ismain_noend = ispre_ismain_noend.str.split(',')
  main_data = pd.concat([nopre_noend, ispre_ismain_noend],axis=0)
  main_data.reset_index(drop=True, inplace=True)
  main_list = []
  for i in range(main_data.shape[0]):
    for j in range(len(main_data[i])):
        main_list.append(main_data[i][j])
  main_list = pd.DataFrame(main_list)

  #마무리운동 리스트 만들기
  ispre_nomain_isend = ispre_nomain_isend.str.split(',')
  ispre_ismain_isend = ispre_ismain_isend.str.split(',')
  nopre_isend = nopre_isend.str.split(',')
  end_data = pd.concat([ispre_nomain_isend, ispre_ismain_isend, nopre_isend], axis=0)
  end_data.reset_index(drop=True, inplace=True)
  end_list = []
  for i in range(end_data.shape[0]):
    for j in range(len(end_data[i])):
        end_list.append(end_data[i][j])
  end_list = pd.DataFrame(end_list)
  
  return pre_list, main_list, end_list

In [15]:
c0_pre, c0_main, c0_end = mvm_list(df_2022_c0)
c1_pre, c1_main, c1_end = mvm_list(df_2022_c1)
c2_pre, c2_main, c2_end = mvm_list(df_2022_c2)
c3_pre, c3_main, c3_end = mvm_list(df_2022_c3)
c4_pre, c4_main, c4_end = mvm_list(df_2022_c4)

In [75]:
#5번 클러스터 - 마무리 없음
df_2022_c5.reset_index(drop=True, inplace=True)
c5_ispre = df_2022_c5['MVM_PRSCRPTN_CN'].str.split('준비운동:',expand=True)[1]
c5_ispre_pre = c5_ispre.str.split(' / 본운동:',expand=True)[0].str.split(',')
c5_ispre_main_end = c5_ispre.str.split(' / 본운동:',expand=True)[1]
c5_ispre_main = c5_ispre_main_end.str.split(' / 마무리운동:',expand=True)[0].str.split(',')

c5_pre = []
for i in range(df_2022_c5.shape[0]):
  for j in range(len(c5_ispre_pre[i])):
    c5_pre.append(c5_ispre_pre[i][j])
c5_pre = pd.DataFrame(c5_pre)

c5_main = []
for i in range(df_2022_c5.shape[0]):
  for j in range(len(c5_ispre_main[i])):
    c5_main.append(c5_ispre_main[i][j])
c5_main = pd.DataFrame(c5_main)

c5_end = []
c5_end = pd.DataFrame(c5_end)

In [76]:
c5_pre.value_counts()

누워서 전신 뻗기         1
동적 스트레칭 루틴프로그램    1
등/어깨 뒤쪽 스트레칭      1
어깨 뒤쪽 스트레칭        1
엉덩이 스트레칭          1
엉덩이 스트레칭2         1
전신 루틴 스트레칭        1
dtype: int64

In [77]:
c5_main.value_counts()

넙다리 안쪽 스트레칭     2
넙다리 앞쪽 스트레칭     2
엉덩이 스트레칭        2
넙다리 뒤쪽 스트레칭     1
등/어깨 뒤쪽 스트레칭    1
상지 루틴 스트레칭      1
하지 루틴 스트레칭1     1
하지 루틴 스트레칭2     1
허리 스트레칭         1
dtype: int64

In [97]:
c5_end.value_counts()

Series([], dtype: int64)

In [89]:
#6번 클러스터
df_2022_c6.dropna(axis=0,inplace=True)
df_2022_c6.reset_index(drop=True, inplace=True)
c6_ispre = df_2022_c6['MVM_PRSCRPTN_CN'].str.split('준비운동:',expand=True)[1]
c6_ispre_pre = c6_ispre.str.split(' / 본운동:',expand=True)[0].str.split(',')
c6_ispre_main_end = c6_ispre.str.split(' / 본운동:',expand=True)[1]
c6_ispre_main = c6_ispre_main_end.str.split(' / 마무리운동:',expand=True)[0].str.split(',')
c6_ispre_end = c6_ispre_main_end.str.split(' / 마무리운동:',expand=True)[1].str.split(',')

c6_pre = []
for i in range(df_2022_c6.shape[0]):
  for j in range(len(c6_ispre_pre[i])):
    c6_pre.append(c6_ispre_pre[i][j])
c6_pre = pd.DataFrame(c6_pre)

c6_main = []
for i in range(df_2022_c6.shape[0]):
  for j in range(len(c6_ispre_main[i])):
    c6_main.append(c6_ispre_main[i][j])
c6_main = pd.DataFrame(c6_main)

c6_end = []
for i in range(df_2022_c6.shape[0]):
  for j in range(len(c6_ispre_end[i])):
    c6_end.append(c6_ispre_end[i][j])
c6_end = pd.DataFrame(c6_end)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [93]:
c6_pre.value_counts()

뒤로 팔굽혀펴기    1
팔굽혀 펴기      1
dtype: int64

In [94]:
c6_main.value_counts()

누워서 팔 밀기      1
앉아서 가슴 모으기    1
앉아서 당겨 내리기    1
앉아서 양팔 당기기    1
팔다리 교차 버티기    1
dtype: int64

In [95]:
c6_end.value_counts()

뒤로 팔 뻗기         1
팔꿈치 고정 후 벌리기    1
dtype: int64